In [9]:
pip uninstall argostranslate ctranslate2 -y


Found existing installation: argostranslate 1.9.6
Uninstalling argostranslate-1.9.6:
  Successfully uninstalled argostranslate-1.9.6
Found existing installation: ctranslate2 4.6.0
Uninstalling ctranslate2-4.6.0:
  Successfully uninstalled ctranslate2-4.6.0
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install argostranslate


  Using cached argostranslate-1.9.6-py3-none-any.whl.metadata (10 kB)
  Using cached ctranslate2-4.6.0-cp312-cp312-win_amd64.whl.metadata (10 kB)
Using cached argostranslate-1.9.6-py3-none-any.whl (34 kB)
Using cached ctranslate2-4.6.0-cp312-cp312-win_amd64.whl (19.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
import argostranslate.package
import argostranslate.translate

# تحميل ملف ترجمة بصيغة .argosmodel (من الموقع مثلاً)
argostranslate.package.install_from_path("en_ar.argosmodel")

# ترجمة
translated_text = argostranslate.translate.translate("hello", "en", "ar")
print(translated_text)


Exception: Not a valid Argos Model (must be a zip archive)

In [2]:
# التثبيت (شغليهم مرة واحدة فقط)
# !pip install argostranslate==1.8.0 pyttsx3 pyperclip langdetect

import argostranslate.package
import argostranslate.translate
import urllib.request
import pyttsx3
import pyperclip
import os
import tempfile
from langdetect import detect

# --- تحميل وتثبيت نموذج الترجمة تلقائيًا ---
def install_translation_package(from_code, to_code):
    url = "https://www.argosopentech.com/argospm/index.json"
    response = urllib.request.urlopen(url)
    packages = response.read().decode("utf-8")

    import json
    packages = json.loads(packages)
    for pkg in packages:
        if pkg["from_code"] == from_code and pkg["to_code"] == to_code:
            model_url = pkg["package_path"]
            with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
                print(f"📥 جاري تحميل نموذج الترجمة: {from_code} → {to_code}...")
                urllib.request.urlretrieve(model_url, tmp_file.name)
                argostranslate.package.install_from_path(tmp_file.name)
                print("✅ النموذج تم تثبيته.")
            break

# --- تحميل الحزم المثبتة ---
def get_installed_languages():
    return argostranslate.translate.get_installed_languages()

# --- عرض اللغات المتاحة أوفلاين ---
installed_languages = get_installed_languages()
language_names = [lang.name for lang in installed_languages]
print("📚 اللغات المدعومة حالياً (أوفلاين):")
print(", ".join(language_names))

# --- إدخال المستخدم ---
text = input("\n📝 Enter text to translate: ")
target_lang_name = input("🌐 Enter target language (e.g., Arabic): ").capitalize()

# --- تحديد اللغة الأصلية تلقائيًا ---
source_lang_code = detect(text)
source_lang_obj = next((lang for lang in installed_languages if lang.code == source_lang_code), None)
target_lang_obj = next((lang for lang in installed_languages if lang.name == target_lang_name), None)

# --- لو النموذج مش موجود، نحاول نحمله تلقائيًا
if not (source_lang_obj and target_lang_obj):
    print("🔍 جاري البحث عن نموذج الترجمة المناسب...")
    install_translation_package(source_lang_code, target_lang_obj.code if target_lang_obj else "ar")
    installed_languages = get_installed_languages()
    source_lang_obj = next((lang for lang in installed_languages if lang.code == source_lang_code), None)
    target_lang_obj = next((lang for lang in installed_languages if lang.name == target_lang_name), None)

# --- الترجمة ---
try:
    if source_lang_obj and target_lang_obj:
        translation = source_lang_obj.get_translation(target_lang_obj)
        translated = translation.translate(text)
        print("\n✅ Translated Text:")
        print(translated)

        # نسخ الترجمة
        do_copy = input("\n📋 Do you want to copy the translation? (y/n): ")
        if do_copy.lower() == "y":
            pyperclip.copy(translated)
            print("✔️ Copied to clipboard!")

        # نطق صوتي
        do_speak = input("\n🔊 Do you want to hear the translation? (y/n): ")
        if do_speak.lower() == "y":
            engine = pyttsx3.init()
            engine.say(translated)
            engine.runAndWait()
    else:
        print("❌ لم يتم العثور على النموذج بين اللغتين.")
except Exception as e:
    print(f"❌ Error: {e}")


HTTPError: HTTP Error 404: Not Found